# 5. Record an original video of a subject in a stationary background                  (No green screens).    a ) Change the original background with another background.    b ) Show the optical flow of the video

In [6]:
import cv2 as cv
import numpy as np
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation

# initialize video capture object
vid = cv.VideoCapture(0)

# set width and height of the video capture object
vid.set(3, 640) #width
vid.set(4, 480) #height

# load background image
bg_img = cv.imread("background.jpg")

# create selfi segmentation object
seg = SelfiSegmentation()

# loop through each frame of the video
while True:
    # read the frame from the video capture object
    _, video = vid.read()
    if not _:
        break
    
    # remove background using the selfi segmentation object
    vid_rmbg = seg.removeBG(video, bg_img, threshold=0.8)
    
    # display the processed video frame
    cv.imshow("Video", vid_rmbg)
    
    # break the loop if 'x' key is pressed
    if cv.waitKey(1) == ord('x'):
        break
        
# close all windows
cv.destroyAllWindows()

# initialize video capture object
cap = cv.VideoCapture(0)

# read the first frame from the video capture object and convert it to grayscale
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

# initialize an array for storing the HSV values
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

# loop through each frame of the video
while(1):
    # read the frame from the video capture object
    ret, frame2 = cap.read()
    
    # break the loop if no frame is grabbed
    if not ret:
        print('No frames grabbed!')
        break
    
    # convert the current frame to grayscale
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    
    # calculate the optical flow between the previous and current frames
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # convert the optical flow vectors to polar coordinates
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    
    # convert the angle values from radians to degrees and scale the magnitude values
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    
    # convert the HSV values to BGR and display the processed frame
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    
    # break the loop if 'Esc' key is pressed
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
        
    # save the current frame as an image if 's' key is pressed
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    
    # update the previous frame
    prvs = next

# close all windows
cv.destroyAllWindows()
